## Data Storage with NetCDF4 and HDF5

In [ ]:
%matplotlib inline

In [ ]:
#required imports
import numpy as np
import h5py
import netCDF4 as nc
import xarray as xr

#### Basic HDF5 usage and comparison to binary

In [ ]:
# Let's create a 500 x 500 array of floats:
test_arr = np.random.rand(500, 500)
test_arr

Let's try to save this 500 x 500 grid as pure binary

In [ ]:
!mkdir ./test_2/

In [ ]:
# save as comma separated
test_arr.tofile(open("./test_2/large_out_arr_binary.binary", "wb"), sep='')

In [ ]:
!ls -lh ./test_2

Now, let's try to save it as a regular HDF5 file. HDF5 files contain some metadata, but by default require very little.

In [ ]:
hf = h5py.File("./test_2/large_out_arr.h5", "w")

In [ ]:
hf.create_dataset('test_arr', data=test_arr)

In [ ]:
hf.close()

In [ ]:
!ls -l ./test_2

In [ ]:
hf_r = h5py.File("./test_2/large_out_arr.h5", "r")
hf_r["test_arr"]

Notice the *lazy* loading- when we look through the variables, it doesn't require us to load in the array. 

In [ ]:
test_arr_reloaded = np.array(hf_r["test_arr"])
np.all(test_arr == test_arr_reloaded)

In [ ]:
hf_r.close()

### Lossless Compression

In [ ]:
hf = h5py.File("./test_2/large_out_arr_comp_gzip.h5", 'w')
hf.create_dataset('test_arr', data=test_arr, compression="gzip", compression_opts=6)
hf.close()


In [ ]:
!ls -l ./test_2

In [ ]:
hf = h5py.File("./test_2/large_out_arr_comp_lzf.h5", 'w')
hf.create_dataset('test_arr', data=test_arr, compression="lzf",  shuffle=True)
hf.close()


In [ ]:
!ls -l ./test_2

In [ ]:
hf = h5py.File("./test_2/large_out_arr_comp_shuffle.h5", 'w')
hf.create_dataset('test_arr', data=test_arr, compression="gzip", compression_opts=6, shuffle=True)
hf.close()


In [ ]:
!ls -l ./test_2

### Lossy Compression

In [ ]:
hf = h5py.File("./test_2/large_out_arr_comp_lossy_scaleoffset.h5", 'w')
hf.create_dataset('test_arr', data=test_arr, compression="gzip", compression_opts=6, shuffle=True, scaleoffset=3)
hf.close()


In [ ]:
!ls -l ./test_2/

In [ ]:
from scipy import signal
# https://stackoverflow.com/a/46892763
def gkern(kernlen=21, std=3):
    """Returns a 2D Gaussian kernel array."""
    gkern1d = signal.windows.gaussian(kernlen, std=std).reshape(kernlen, 1)
    gkern2d = np.outer(gkern1d, gkern1d)
    return gkern2d


In [ ]:
test_arr_gauss = np.array(gkern(kernlen=200, std=50))*50

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.pcolormesh(test_arr_gauss, vmin=0, vmax=50)
plt.colorbar()

In [ ]:
# save lossy
hf = h5py.File("./test_2/gauss_comp_lossy_scaleoffset_3.h5", 'w')
hf.create_dataset('test_arr_gauss', data=test_arr_gauss, compression="gzip", compression_opts=6, shuffle=True, scaleoffset=3)
hf.close()


In [ ]:
hf_r = h5py.File("./test_2/gauss_comp_lossy_scaleoffset_3.h5", 'r')
test_arr_reloaded = np.array(hf_r["test_arr_gauss"])
np.all(test_arr_gauss == test_arr_reloaded)

In [ ]:
plt.pcolormesh(test_arr_reloaded, vmin=0, vmax=50)
plt.colorbar()

In [ ]:
plt.pcolormesh(test_arr_reloaded-test_arr_gauss, vmin=-0.01, vmax=0.01, cmap='bwr')
plt.colorbar()

In [ ]:
hf_r.close()